#**Autocorrect Tool**

**Basic description:**
The notebook is designed to create a simple autocorrect tool that suggests correct spellings for misspelled words based on a vocabulary and word probabilities. The tool uses two levels of edits (single-character edits and two-character edits) to generate potential corrections.

In [1]:
import string
import re
import numpy as np
from collections import Counter

Going throungh the data

In [2]:
def read_corpus(filename):
    with open(filename,'r',encoding='utf-8') as file:
        lines = file.readlines()

        words = []
        for word in lines:
            words += re.findall(r'\w+',word.lower())
    return words

corpus = read_corpus(r'big.txt')

In [3]:
len(corpus)

222663

Build our lexicon of unique words

In [4]:
vocab = set(corpus)
len(vocab)

17647

Count the occurrences of words in our corpus

In [5]:
words_count = Counter(corpus)
words_count

Counter({'the': 14703,
         'project': 91,
         'gutenberg': 94,
         'ebook': 10,
         'of': 6742,
         'moby': 90,
         'dick': 90,
         'or': 797,
         'whale': 1230,
         'by': 1222,
         'herman': 4,
         'melville': 4,
         'this': 1439,
         'is': 1751,
         'for': 1644,
         'use': 49,
         'anyone': 6,
         'anywhere': 16,
         'at': 1335,
         'no': 594,
         'cost': 4,
         'and': 6517,
         'with': 1769,
         'almost': 197,
         'restrictions': 2,
         'whatsoever': 7,
         'you': 958,
         'may': 255,
         'copy': 19,
         'it': 2534,
         'give': 90,
         'away': 186,
         're': 18,
         'under': 126,
         'terms': 33,
         'license': 18,
         'included': 14,
         'online': 4,
         'www': 6,
         'org': 13,
         'title': 8,
         'author': 9,
         'release': 1,
         'date': 4,
         'december': 5,
   

In [6]:
total_words_count = float(sum(words_count.values()))

In [7]:
word_probabs = {word:words_count[word] / total_words_count for word in words_count.keys()}

In [8]:
word_probabs['the']

0.06603252448767868

Split, delete, swap, replace and insert operations:


In [9]:
def split(word): # why
    return [ (word[:i], word[i:])  for i in range(len(word) + 1)]

In [10]:
print(split('why'))

[('', 'why'), ('w', 'hy'), ('wh', 'y'), ('why', '')]


In [11]:
def delete(word):
    return [left + right[1:] for left,right in split(word) if right]

In [12]:
print(delete('why'))

['hy', 'wy', 'wh']


In [13]:
def swap(word):
    return [left + right[1] + right[0] + right[2:] for left,right in split(word) if len(right) > 1 ]

In [14]:
print(swap('why')) #hy == yh

['hwy', 'wyh']


In [15]:
def replace(word): # abcdef...z
    return [left + center + right[1:] for left, right in split(word) if right for center in string.ascii_lowercase]

In [16]:
print(replace('why'))

['ahy', 'bhy', 'chy', 'dhy', 'ehy', 'fhy', 'ghy', 'hhy', 'ihy', 'jhy', 'khy', 'lhy', 'mhy', 'nhy', 'ohy', 'phy', 'qhy', 'rhy', 'shy', 'thy', 'uhy', 'vhy', 'why', 'xhy', 'yhy', 'zhy', 'way', 'wby', 'wcy', 'wdy', 'wey', 'wfy', 'wgy', 'why', 'wiy', 'wjy', 'wky', 'wly', 'wmy', 'wny', 'woy', 'wpy', 'wqy', 'wry', 'wsy', 'wty', 'wuy', 'wvy', 'wwy', 'wxy', 'wyy', 'wzy', 'wha', 'whb', 'whc', 'whd', 'whe', 'whf', 'whg', 'whh', 'whi', 'whj', 'whk', 'whl', 'whm', 'whn', 'who', 'whp', 'whq', 'whr', 'whs', 'wht', 'whu', 'whv', 'whw', 'whx', 'why', 'whz']


In [17]:
def insert(word): # abcdef...z
    return [left + center + right[1:] for left, right in split(word) for center in string.ascii_lowercase]

In [18]:
print(replace('love'))

['aove', 'bove', 'cove', 'dove', 'eove', 'fove', 'gove', 'hove', 'iove', 'jove', 'kove', 'love', 'move', 'nove', 'oove', 'pove', 'qove', 'rove', 'sove', 'tove', 'uove', 'vove', 'wove', 'xove', 'yove', 'zove', 'lave', 'lbve', 'lcve', 'ldve', 'leve', 'lfve', 'lgve', 'lhve', 'live', 'ljve', 'lkve', 'llve', 'lmve', 'lnve', 'love', 'lpve', 'lqve', 'lrve', 'lsve', 'ltve', 'luve', 'lvve', 'lwve', 'lxve', 'lyve', 'lzve', 'loae', 'lobe', 'loce', 'lode', 'loee', 'lofe', 'loge', 'lohe', 'loie', 'loje', 'loke', 'lole', 'lome', 'lone', 'looe', 'lope', 'loqe', 'lore', 'lose', 'lote', 'loue', 'love', 'lowe', 'loxe', 'loye', 'loze', 'lova', 'lovb', 'lovc', 'lovd', 'love', 'lovf', 'lovg', 'lovh', 'lovi', 'lovj', 'lovk', 'lovl', 'lovm', 'lovn', 'lovo', 'lovp', 'lovq', 'lovr', 'lovs', 'lovt', 'lovu', 'lovv', 'lovw', 'lovx', 'lovy', 'lovz']


In [27]:
#Min-distance
def level_1(word):
    return set((delete(word) + swap(word) + replace(word) + insert(word)))

In [28]:
print(level_1('load'))

{'loay', 'loadx', 'loadf', 'lpad', 'load', 'lofd', 'lcad', 'loadu', 'loadt', 'logd', 'lotd', 'xoad', 'koad', 'lyad', 'llad', 'loid', 'lmad', 'ioad', 'lxad', 'loar', 'lad', 'lvad', 'loaz', 'loak', 'lkad', 'goad', 'oad', 'lohd', 'road', 'lbad', 'lojd', 'lovd', 'lzad', 'uoad', 'noad', 'loan', 'ltad', 'loadr', 'loag', 'lozd', 'loax', 'lod', 'loed', 'yoad', 'foad', 'lokd', 'lopd', 'ljad', 'loae', 'loam', 'loaw', 'ooad', 'lrad', 'loaj', 'loadw', 'soad', 'lord', 'toad', 'lfad', 'loaf', 'loadc', 'loadg', 'ldad', 'loyd', 'qoad', 'voad', 'zoad', 'lhad', 'loade', 'loadk', 'lnad', 'eoad', 'loav', 'doad', 'loadq', 'loada', 'loadj', 'loab', 'losd', 'laad', 'loal', 'poad', 'loadm', 'lobd', 'boad', 'loado', 'aoad', 'loady', 'liad', 'loah', 'woad', 'loadp', 'lood', 'lomd', 'loadb', 'lodd', 'loap', 'lgad', 'loadd', 'coad', 'loaa', 'loai', 'loaq', 'loqd', 'loadi', 'lead', 'loadz', 'loac', 'loa', 'olad', 'hoad', 'loud', 'loda', 'joad', 'lold', 'lond', 'loxd', 'laod', 'loadv', 'loat', 'lsad', 'locd', 'lowd

In [31]:
def level_2(word):
    return set(e2  for e1 in level_1(word) for e2 in level_1(e1))

In [32]:
print(level_2('cut'))

{'cutld', 'cuttd', 'outk', 'hht', 'outt', 'cues', 'cuxl', 'cutgx', 'cut', 'cdto', 'cqc', 'gutj', 'xux', 'jutm', 'cuhk', 'iutz', 'duth', 'cunz', 'ukt', 'ajt', 'cuc', 'juta', 'cubr', 'pu', 'cstc', 'cujz', 'cuzr', 'muk', 'cutzd', 'cutlp', 'lui', 'dutf', 'cutbg', 'cumz', 'cgm', 'kutp', 'cutoi', 'puc', 'njt', 'cmtw', 'cuna', 'cqg', 'cutig', 'gutg', 'iue', 'cvi', 'net', 'cuttl', 'cudh', 'cutsm', 'chl', 'juu', 'cstx', 'cgv', 'bet', 'cudn', 'cstw', 'cutpg', 'lutx', 'cutsj', 'uctc', 'sua', 'kua', 'cutle', 'cetj', 'cgte', 'iutn', 'catq', 'cdtx', 'cvtk', 'cbto', 'puz', 'cunq', 'cugh', 'cupz', 'cugj', 'cutzk', 'twt', 'cuua', 'dt', 'up', 'zrt', 'cye', 'cucu', 'sjt', 'cuyv', 'xwt', 'cptr', 'cuch', 'cutnz', 'outf', 'catx', 'yxt', 'cunw', 'aat', 'futn', 'cuqr', 'cmk', 'catc', 'cupt', 'wuk', 'nuth', 'guw', 'zdt', 'cugi', 'opt', 'mkt', 'hmt', 'co', 'eht', 'kyt', 'cttt', 'cfw', 'cutqn', 'citj', 'cutrv', 'kst', 'cuyp', 'outi', 'cutqr', 'cutnf', 'hutp', 'yuti', 'eug', 'cnm', 'czg', 'buq', 'xuh', 'cgn', 'cj

Autocorrect Query Interface

In [33]:
def correct_spelling(word,vocab,word_probabs):
    if word in vocab:
        print(f"{word} is already correctly spelled")
        return
    #getting all suggesions
    suggestions = level_1(word) or level_2(word) or [word]
    best_guesses = [w for w in suggestions if w in vocab]
    return [(w, word_probabs[w]) for w in best_guesses]

In [25]:
search_word = "cunt"
guess = correct_spelling(search_word,vocab,word_probabs)
print(guess)

[('cut', 0.0002380278717164504), ('aunt', 1.796436767671324e-05), ('cent', 4.49109191917831e-06), ('cant', 1.347327575753493e-05), ('hunt', 0.00010778620606027944)]
